### Import reviews

In [42]:
%load_ext autoreload
%autoreload 2

import progressbar

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    labels = pickle.load(f)
   
entity_names = ['X', 'Y', 'Z']

In [44]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,labels,balanced=True, shuffled=True, seed=456)

Grammar construction complete.


In [ ]:
from babble.utils import display_candidate

candidate = babbler.next()
print(candidate)
print(candidate.text)
print(candidate.entity_positions)

In [45]:
import re

def get_words_before(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity)
        before = index - min(index, quantity)
        return ' '.join(map(str, words[before:index])) 

In [46]:
import re

def get_words_after(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity) +1
        after = index + min(index, quantity)
        return ' '.join(map(str, words[index:after]))

In [47]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

pos_adjectives_list=[]
neu_adjectives_list=[]
neg_adjectives_list=[]

def check_sentiment_adjective(adjective):
    if (sid.polarity_scores(adjective)['compound']) >= 0.1:
        pos_adjectives_list.append(adjective)
        return 'positive'
    elif (sid.polarity_scores(adjective)['compound']) <= -0.1:
        neg_adjectives_list.append(adjective)
        return 'negative'
    else:
        neu_adjectives_list.append(adjective)
        return 'neutral' 

In [ ]:
check_sentiment_adjective('normal')

In [48]:
def check_adjectives_before_pos(sentence,entity):
    words = get_words_before(2,sentence,entity)
    if words == None or len(words.split(" ")) == 0:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        for token in doc:
            if token.pos_ == 'ADJ':
                return token.text
                break

In [49]:
def check_adjectives_after_verb(sentence,entity):
    words = get_words_after(3,sentence,entity)
    if words == None or len(words.split(" ")) < 3:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        if doc[0].pos_ == 'VERB' and (doc[1].pos_ == 'ADJ' or doc[2].pos_ == 'ADJ'):
            return doc[1].text

In [ ]:
def create_condition()

In [ ]:
print(get_words_before(2,candidate.text,"chicken"))

In [ ]:
print(check_adjectives_before_pos(candidate.text,"customer"))

In [50]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

spaCy Version: 2.1.8


In [51]:
from babble import Explanation

explanations = []
index = 1


for candidate,label in progressbar.progressbar(zip(reviews[0],labels[0])):
#    print(candidate.text)
    for entity,name in zip(candidate.entities,entity_names):
#        print(entity.entity)
        adjective = check_adjectives_after_verb(candidate.text,entity.entity)
        if adjective != None:
            sentiment_value = check_sentiment_adjective(adjective)
            condition='A ' + sentiment_value + ' word is within three words to the right of "' + entity.entity + '"'
        else:
            adjective = check_adjectives_before_pos(candidate.text,entity.entity)
            if adjective != None:
                sentiment_value = check_sentiment_adjective(adjective)
                condition='A ' + sentiment_value + ' word is within two words to the left of "' + entity.entity + '"'
        if adjective != None and sentiment_value != 'neutral':
            explanation = Explanation(
            name='LF_' + str(index),
            label=label,
            condition=condition,
            candidate=candidate,
            )
            explanations.append(explanation)
            index = index + 1
#            print(str(explanation).upper())

| |               #                                | 9356 Elapsed Time: 2:35:20


In [25]:
for explanation in explanations:
    print(explanation.candidate)

RelationMention(doc_id=1: entities=("total"(0:5), "bill"(6:10), "service"(29:36))
RelationMention(doc_id=3: entities=("place"(6:11), "staff"(13:18), "help"(71:75))
RelationMention(doc_id=8: entities=("hour"(27:31), "food"(56:60), "bread"(107:112))
RelationMention(doc_id=13: entities=("service"(6:13), "drink"(66:71), "counter"(98:105))
RelationMention(doc_id=16: entities=("good"(0:4), "luck"(5:9), "appointment"(21:32))
RelationMention(doc_id=19: entities=("nachos"(18:24), "meat"(30:34), "water"(113:118))
RelationMention(doc_id=20: entities=("customer"(9:17), "service"(18:25), "overprice"(28:37))
RelationMention(doc_id=20: entities=("customer"(9:17), "service"(18:25), "overprice"(28:37))
RelationMention(doc_id=25: entities=("thing"(19:24), "restaurant"(31:41), "food"(45:49))
RelationMention(doc_id=27: entities=("gym"(9:12), "ceiling"(18:25), "shower"(38:44))
RelationMention(doc_id=28: entities=("country"(18:25), "chicago"(72:79), "location"(128:136))
RelationMention(doc_id=28: entities=(

In [ ]:
print(explanations)

In [ ]:
babbler.view_parse(parses[1])

In [ ]:
print(parses[10].semantics)

In [ ]:
parses, filtered = babbler.apply(explanations)

In [ ]:
babbler.commit()

In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], labels[1], lf_names=lf_names)

In [ ]:
print(Ls)

In [52]:
pos_adjectives_list = list(dict.fromkeys(pos_adjectives_list))
neu_adjectives_list = list(dict.fromkeys(neu_adjectives_list))
neg_adjectives_list = list(dict.fromkeys(neg_adjectives_list))

In [53]:
with open(DATA_FILE3, 'wb') as f:
    pickle.dump(pos_adjectives_list,f)
    
with open(DATA_FILE4, 'wb') as f:
    pickle.dump(neu_adjectives_list,f)

with open(DATA_FILE5, 'wb') as f:
    pickle.dump(neg_adjectives_list,f)

In [54]:
from babble.utils import ExplanationIO

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)

Wrote 5199 explanations to data/my_explanations.tsv
Read 5199 explanations from data/my_explanations.tsv
